In [44]:
import pandas as pd

data = pd.read_csv('Data\\tweet_results_4_11.csv', parse_dates=['created_at'])
data = data[data['language'] == 'en']
data = data.sort_values(by='created_at')
data.head()

C:\Users\luis.lavierihernande\Anaconda3\envs\py37\lib\site-packages\dateutil\parser\_parser.py:1218: UnknownTimezoneWarning: tzname EST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


,created_at,tweet,language,mentions,urls,photos,replies_count,retweets_count,likes_count,hashtags,cashtags
1566,2020-12-19 19:32:52,@bigtitstrader XLM cause I like the way they n...,en,[],[],[],1,0,2,[],[]
1565,2020-12-20 00:45:32,"If you're doing Coinbases ""earn rewards"": Up ...",en,[],['https://coinbase.com/earn/comp/invite/y5c9fs...,[],1,1,2,[],[]
1564,2020-12-20 03:11:05,Just remembered about my XLM Stellar Lumens wa...,en,[],[],[],0,0,1,"['xlm', 'crypto', 'stellarlumens']",[]
1562,2020-12-20 04:47:57,Stellar Lumens free #cryptocurrency #StellarLu...,en,[],['https://coinbase.com/earn/xlm/invite/4t038b9c'],[],0,0,0,"['cryptocurrency', 'stellarlumens']",[]
1561,2020-12-20 05:52:27,@PRX0202 @kaboom12344 I think the stars are ab...,en,[],[],[],0,0,2,[],[]


In [45]:
import json
import contractions
import string
import re

table = str.maketrans(dict.fromkeys(string.punctuation))
contractions.add('shoulda', 'should have')

regrex_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)

def fixer(row):
    tweet = row['tweet'].lower()
    for mention in json.loads(str(row['mentions']).replace('\'', '"')):
        tweet = tweet.replace(str('@') + str(mention['screen_name']), '')
    for hashtag in json.loads(str(row['hashtags']).replace('\'', '"')):
        tweet = tweet.replace(str('#') + hashtag, '') 
    for cashtag in json.loads(str(row['cashtags']).replace('\'', '"')):
        tweet = tweet.replace(str('#') + cashtag, '')

    try:
        tweet = contractions.fix(tweet)
    except:
        tweet = tweet

    tweet = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', tweet)
    tweet = re.sub(r'@[\d|\w]*', '', tweet)
    tweet = re.sub(r'#[\d|\w]*', '', tweet)
    tweet = regrex_pattern.sub(r'',tweet)

    tweet = tweet.lower().translate(table)
    tweet = ' '.join(tweet.split())
    
    
    return tweet

data['tweet'] = [fixer(row) for ind, row in data.iterrows()]

In [47]:
data = data.drop('language', 1)
data = data.drop('mentions', 1)
data = data.drop('urls', 1)
data = data.drop('photos', 1)
data = data.drop('cashtags', 1)
data = data.drop('hashtags', 1)

data.head()
data.to_csv('Data\\tweet_results_4_11_2.csv')